In [27]:
# Get the openai secret key:
import getpass

secret_key = getpass.getpass('Please enter your openai key:')

In [ ]:
%pip install wikipedia

In [28]:
# 1. Standard Tools
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

print(tool.name)
print(tool.description)
print(tool.args)

# We can see if the tool should return directly to the user
print("Will this automatically return the output to the user? This value is a boolean:", tool.return_direct)

wikipedia
A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
{'query': {'description': 'query to look up on wikipedia', 'title': 'Query', 'type': 'string'}}
Will this automatically return the output to the user? This value is a boolean: False


In [29]:
tool.invoke("What is Java Engineering?")

'Page: Java (programming language)\nSummary: Java is a high-level, general-purpose, memory-safe, object-oriented programming language. It is intended to let programmers write once, run anywhere (WORA), meaning that compiled Java code can run on all platforms that support Java without the need to recompile. Java applications are typically compiled to bytecode that can run on any Java virtual machine (JVM) regardless of the underlying computer architecture. The syntax of Java is similar to C and C++, but has fewer low-level facilities than either of them. The Java runtime provides dynamic capabilities (such as reflection and runtime code modification) that are typically not available in traditional compiled languages.\nJava gained popularity shortly after its release, and has been a popular programming language since then. Java was the third most popular programming language in 2022 according to GitHub. Although still widely popular, there has been a gradual decline in use of Java in rec

In [31]:
# Import things that are needed generically
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [ ]:
@tool
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

print(search.name)
print(search.description)
print(search.args)

In [ ]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")

@tool("search-tool", args_schema=SearchInput, return_direct=True)
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

In [8]:
def search_function(query: str):
    return "LangChain"

search = StructuredTool.from_function(
    func=search_function,
    name="Search",
    description="useful for when you need to answer questions about current events",
    # you can specify an async method if desired as well
)

In [9]:
from langchain.agents import tool

# 1. Create the tool:
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

# 2. Assign the tools to a Python list:
tools = [get_word_length]

In [12]:
# 3. Create the ChatPromptTemplate:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [32]:
from langchain_openai import ChatOpenAI

# 4. Create the LLM and bind the tools directly to the LLM:
llm = ChatOpenAI(model='gpt-4o-mini', api_key=secret_key)
llm_with_tools = llm.bind_tools(tools=tools)

In [33]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

# 5. Creating the LCEL agent chain:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [34]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, openai_api_key=os.environ["OPENAI_API_KEY"])

In [35]:
list(agent_executor.stream( {"input": "How many letters in the word data"}))



> Entering new None chain...

Invoking: `get_word_length` with `{'word': 'data'}`


4The word "data" has 4 letters.

> Finished chain.


[{'actions': [ToolAgentAction(tool='get_word_length', tool_input={'word': 'data'}, log="\nInvoking: `get_word_length` with `{'word': 'data'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_k4HBhG9c8tXQp0h0iXR15dw8', 'function': {'arguments': '{"word":"data"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_3267753c5d'}, id='run-c517d3e0-5d0a-42f3-85cf-6f1f36053d6f', tool_calls=[{'name': 'get_word_length', 'args': {'word': 'data'}, 'id': 'call_k4HBhG9c8tXQp0h0iXR15dw8', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_word_length', 'args': '{"word":"data"}', 'id': 'call_k4HBhG9c8tXQp0h0iXR15dw8', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_k4HBhG9c8tXQp0h0iXR15dw8')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_k4HBhG9c8tXQp0h0iXR

In [36]:
#Adding Memory to the agent

from langchain_core.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [37]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

In [38]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [39]:
input1 = "how many letters in the word data?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'data'}`


4The word "data" has 4 letters.

> Finished chain.


> Entering new AgentExecutor chain...
Yes, "data" is a real word. It refers to factual information used for reference or analysis. The term is commonly used in contexts such as statistics, computing, and research.

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word data?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The word "data" has 4 letters.', additional_kwargs={}, response_metadata={})],
 'output': 'Yes, "data" is a real word. It refers to factual information used for reference or analysis. The term is commonly used in contexts such as statistics, computing, and research.'}

In [53]:
# Customising the memory by 
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name='history'),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = (
    {
        "input": lambda x: x["input"],
        "history": lambda x: x.get("history", []),
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [54]:
with_message_history.invoke(
    {"input": "My name is Marcell", "history": []},
    config={"configurable": {"session_id": "some_session_id"}},
)



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'Marcell'}`


7The length of your name "Marcell" is 7 letters.

> Finished chain.


{'input': 'My name is Marcell',
 'history': [],
 'output': 'The length of your name "Marcell" is 7 letters.'}

In [55]:
with_message_history.invoke(
    {"input": "What is my name?"},
    config={"configurable": {"session_id": "some_session_id"}},
)



> Entering new AgentExecutor chain...
Your name is Marcell.

> Finished chain.


{'input': 'What is my name?',
 'history': [HumanMessage(content='My name is Marcell', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The length of your name "Marcell" is 7 letters.', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is Marcell.'}